In [1]:
import os
import numpy as np
import mne
import sklearn
import matplotlib.pyplot as plt
import pywt
from collections import Counter

from sklearn import svm
from sklearn.model_selection import train_test_split

from AudioOnsetUtils import *

RAW_EOG_CHANNELS = [u'EXG1', u'EXG2', u'EXG3', u'EXG4']
MASTOID_CHANNELS = [u'EXG5', u'EXG6']

In [2]:
## This cell gets filtered data so we can open ICA for Participant PNUM

def get_participant_filtered_data(PNUM):
    data_raw_file = os.path.join('raw_data', 'P' + PNUM + '-raw.fif')
    raw = mne.io.read_raw_fif(data_raw_file)

    # remove mastoid channels if present 
    if MASTOID_CHANNELS[0] in raw.ch_names:
        mne.io.set_eeg_reference(raw.load_data(), MASTOID_CHANNELS, copy=False) # inplace
        raw.drop_channels(MASTOID_CHANNELS)

    # Drop bad channels - in place on raw
    for bad_channel in raw.info['bads']:
        raw.drop_channels(bad_channel)
#         print("dropped: " + bad_channel)

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, exclude=[])

    # bandpass filter - keeping a frequency range between 0.5 (high pass filter) and 30 Hz (low pass filter)
    filtered_data = raw.load_data().filter(0.5, 30, picks=eeg_picks)
    
    return filtered_data

In [3]:
## This cell opens the ICA file for Participant PNUM

def get_ica_data_from_file(filtered_data, PNUM):
    ica = mne.preprocessing.read_ica('ica_data/P' + PNUM + '-ica.fif')

    # apply the transformation
    postica_data = ica.apply(filtered_data, exclude=ica.exclude)
    return postica_data

In [4]:
## Original code from https://github.com/sstober/openmiir-rl-2016/blob/master/openmiir/events.py#L21 and adapted 
KEYSTROKE_BASE_ID = 2000
TIME_INDEX = 0
ID_INDEX = 1

# returns two things, a True if Not Noise boolean and a tuple with three elements:
# 0- this is lyrical (0-2) <- YLABEL
# 1- start time (onsettime + cue duration if condition 1/2, else just onsettime)
# 2- end time (starttime + duration of song without cue)
def get_event_data(event_times_ids, events_index, music_version):
    event_id = event_times_ids[events_index[0]][ID_INDEX]

    if event_id < 1000:
        """
        Event Ids < 1000 are trial labels
        with the last digit indicating the condition
                1 : 'perception',
                2 : 'cued imag',
                3 : 'imag fix cross',
                4 : 'imagination',
        and the remaining digits referring to the stimulus id.
        """
        stimulus_id, condition = decode_event_id(event_id)
        
        stimulus_info = get_vers_stim_dict(music_version)[stimulus_id]
        cue_length = stimulus_info['cue_length']
        song_length = stimulus_info['song_length']
        
        events_index[0] += 1
        
        #print(get_id_to_song_name(stimulus_id))
        
        # get time of audio onset for this stimulus
        while event_times_ids[events_index[0]][ID_INDEX] != 1000:
            #print("Expected an audio onset event but got {}".format(event_times_ids[events_index[0]]))
            events_index[0] += 1
            
        audio_onset_time = event_times_ids[events_index[0]][TIME_INDEX]
        start_time = (audio_onset_time + cue_length) if condition in [1,2] else audio_onset_time
        end_time = start_time + song_length
        
        # move the pointer by 1 to prep next call to get_event_data
        events_index[0] += 1
        
        return True, (stimulus_info["audio_type"], start_time, end_time)
    else:
        # move the pointer by 1 to prep next call to get_event_data
        events_index[0] += 1
        
        return {
            1111: (False, ("noise")),
            KEYSTROKE_BASE_ID: (False, 'imagination failed'),
            KEYSTROKE_BASE_ID+1: (False, 'imagination okay')
        }[event_id]
    
    
# convert event id to stimulus (song num) and condition (1-4)
def decode_event_id(event_id):
    if event_id < 1000:
        stimulus_id = event_id // 10
        condition = event_id % 10
        return stimulus_id, condition
    else:
        return event_id

In [5]:
def get_event_data_list(event_times_ids, p):
    events_data_list = []
    events_index = [0]
    music_version = get_participant_vers(p)

    while len(events_data_list) < 50:
        is_audio, event_data = get_event_data(event_times_ids, events_index, music_version)
        if is_audio:
            # ignore AudioType.NONLYRICAL_LYRICAL (songs that have lyrics but played w/o lyrics)
            if event_data[0] == AudioType.LYRICAL or event_data[0] == AudioType.NON_LYRICAL:
                events_data_list.append(event_data)
    
    return events_data_list

In [6]:
def get_channels_eeg_data(event_matrix, start_time, end_time, ch_names, postica_data):
    # for each channel
    # extract this individual event's eeg data btwn start and endtime
    sampling_freq = postica_data.info['sfreq']

    for i in range(len(ch_names)):
        start_stop_seconds = np.array([start_time, end_time])
        start_sample, stop_sample = (start_stop_seconds * sampling_freq).astype(int)
        
        # returns a tuple, see https://mne.tools/dev/auto_tutorials/raw/10_raw_overview.html
        #print(postica_data[i, start_sample:stop_sample])
        ch_selection = postica_data[i, :]
        
#         print("==================")
#         print(ch_selection)
        
        # append eeg data list to eventMatrix
        event_matrix.append(ch_selection) 

In [ ]:
## This cell prepares X_list and Y_list (data and labels)

X_list = []
Y_list = []
monster_matrix = [] # list of matrices

#for part in get_participant_list():
for part in ["01","13","14"]:
    # load ica data for participant
    filtered_data = get_participant_filtered_data(part)
    postica_data = get_ica_data_from_file(filtered_data, part)
    
    # from ica data, get stimuli events
    events = mne.find_events(postica_data)

    # creates tuples of (event_time, event_id)
    event_times_ids = [(events[:][i][0], events[:][i][2]) for i in range(len(events))]
    
    # get a list of event datas (enum, starttime, endtime)
    events_list = get_event_data_list(event_times_ids, part)
    
    # for each event in event datas
    for audio_type,start_time,end_time in events_list:
        event_matrix = [] # list of lists

        # get eeg data for all 62 channels, put into event_matrix
        eeg_ch_names = postica_data.ch_names[:62]
        get_channels_eeg_data(event_matrix, start_time, end_time, eeg_ch_names, postica_data)

        # call normalize on event_matrix
      
        # append event_matrix to monster_matrix
        monster_matrix.append(event_matrix)
#         print("EVENT")
#         print(event_matrix)
        
        # 0 for non-lyrical, 1 for lyrical
        Y_list.append(1) if audio_type == AudioType.LYRICAL else Y_list.append(0)

Opening raw data file raw_data\P01-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2478165 =      0.000 ...  4840.166 secs
Ready.
Removing projector <Projection | Average EEG reference, active : False, n_channels : 64>
dropped: P8
dropped: P10
dropped: T8
Reading 0 ... 2478165  =      0.000 ...  4840.166 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 3381 samples (6.604 sec)

Reading ica_data/P01-ica.fif ...
Isot

(array([[-1.41231304e-09, -1.66248497e-06, -3.29649942e-06, ...,
         1.53606515e-06,  7.90617124e-07, -1.41231304e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.32568807e-09, -1.56117592e-06, -3.32793663e-06, ...,
         1.55337633e-06,  8.07686539e-07, -1.32568807e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.29249670e-09, -7.76971012e-07, -1.59139682e-06, ...,
         1.00413609e-06,  5.28104891e-07, -1.29249670e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.31505188e-09, -7.67401751e-07, -1.47059433e-06, ...,
         1.21358226e-06,  6.20967392e-07, -1.31505188e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.59492027e-09

(array([[-2.34928972e-09,  7.29053232e-07,  1.48825282e-06, ...,
        -2.56366752e-06, -1.32346104e-06, -2.34928972e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.88842974e-09,  7.35842852e-07,  1.46880941e-06, ...,
        -8.11926651e-07, -4.06363553e-07, -1.88842974e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-2.33494998e-09,  6.79114945e-07,  1.34797928e-06, ...,
        -7.76959188e-07, -3.85799758e-07, -2.33494998e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-2.11339727e-09,  3.84087565e-07,  7.68682797e-07, ...,
        -5.73377787e-07, -2.86172963e-07, -2.11339727e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-2.05709392e-09

(array([[-1.44603711e-09,  2.04191476e-06,  3.98265569e-06, ...,
        -1.95923280e-06, -1.01326189e-06, -1.44603711e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.63382988e-09,  1.15247102e-06,  2.22560121e-06, ...,
        -7.89725274e-07, -4.03113749e-07, -1.63382988e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.54269415e-09,  5.60565595e-07,  1.10820025e-06, ...,
        -1.08501649e-06, -5.61735921e-07, -1.54269415e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.43027358e-09,  1.52541986e-06,  2.91611259e-06, ...,
         6.95847146e-07,  3.30235364e-07, -1.43027358e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.24572024e-09

(array([[ 7.39374292e-10, -8.91948340e-08, -1.00423708e-07, ...,
        -2.75743788e-06, -1.39567951e-06,  7.39374291e-10]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-8.65686339e-10,  5.50545168e-07,  1.11096926e-06, ...,
         6.74623425e-07,  4.10523735e-07, -8.65686341e-10]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.38799095e-09,  5.13260291e-07,  1.02563922e-06, ...,
        -4.71031809e-08, -2.49355510e-09, -1.38799095e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.79751506e-09,  3.72335125e-07,  7.51670832e-07, ...,
        -3.33364192e-07, -1.46194476e-07, -1.79751506e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.37848748e-09

...

(array([[-1.08816781e-09,  2.35694299e-06,  4.55982399e-06, ...,
        -3.52601833e-06, -1.83633151e-06, -1.08816781e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.44603711e-09,  2.04191476e-06,  3.98265569e-06, ...,
        -1.95923280e-06, -1.01326189e-06, -1.44603711e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.63382988e-09,  1.15247102e-06,  2.22560121e-06, ...,
        -7.89725274e-07, -4.03113749e-07, -1.63382988e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.54269415e-09,  5.60565595e-07,  1.10820025e-06, ...,
        -1.08501649e-06, -5.61735921e-07, -1.54269415e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.43027358e-09

(array([[-1.57589768e-09,  4.90908907e-07,  1.00657491e-06, ...,
         7.71253446e-07,  3.99497548e-07, -1.57589769e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.69215720e-09,  3.35798833e-07,  6.33379619e-07, ...,
         3.06617522e-07,  1.58669056e-07, -1.69215720e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.47177902e-09, -1.04087885e-06, -2.01031754e-06, ...,
         8.60957928e-07,  4.43952007e-07, -1.47177902e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.41231304e-09, -1.66248497e-06, -3.29649942e-06, ...,
         1.53606515e-06,  7.90617124e-07, -1.41231304e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.32568807e-09

(array([[-1.54269415e-09,  5.60565595e-07,  1.10820025e-06, ...,
        -1.08501649e-06, -5.61735921e-07, -1.54269415e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.43027358e-09,  1.52541986e-06,  2.91611259e-06, ...,
         6.95847146e-07,  3.30235364e-07, -1.43027358e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.24572024e-09,  3.68283619e-06,  7.15738292e-06, ...,
        -4.43087166e-06, -2.25357213e-06, -1.24572025e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.12985132e-09,  2.32530355e-06,  4.54515530e-06, ...,
        -4.04335622e-06, -2.07492738e-06, -1.12985132e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.33712366e-09

(array([[-1.47177902e-09, -1.04087885e-06, -2.01031754e-06, ...,
         8.60957928e-07,  4.43952007e-07, -1.47177902e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.41231304e-09, -1.66248497e-06, -3.29649942e-06, ...,
         1.53606515e-06,  7.90617124e-07, -1.41231304e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.32568807e-09, -1.56117592e-06, -3.32793663e-06, ...,
         1.55337633e-06,  8.07686539e-07, -1.32568807e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.29249670e-09, -7.76971012e-07, -1.59139682e-06, ...,
         1.00413609e-06,  5.28104891e-07, -1.29249670e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.31505188e-09

(array([[-1.24572024e-09,  3.68283619e-06,  7.15738292e-06, ...,
        -4.43087166e-06, -2.25357213e-06, -1.24572025e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.12985132e-09,  2.32530355e-06,  4.54515530e-06, ...,
        -4.04335622e-06, -2.07492738e-06, -1.12985132e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.33712366e-09,  5.75662656e-07,  1.13015653e-06, ...,
        -8.74674702e-07, -4.51280787e-07, -1.33712366e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-1.33171596e-09,  4.68060708e-07,  8.25477581e-07, ...,
        -3.86907773e-07, -1.76908481e-07, -1.33171596e-09]]), array([0.00000000e+00, 1.95312500e-03, 3.90625000e-03, ...,
       4.84016211e+03, 4.84016406e+03, 4.84016602e+03]))
(array([[-0.01722228, -0

### Feature Extraction

In [ ]:
# code adapted from https://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning/

def calculate_entropy(list_values):
    counter_values = Counter(tuple(list_values)).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy
 
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]
 
def calculate_crossings(list_values):
    num_zero_crossings = ((val_array[:-1] * val_array[1:]) < 0).sum()
    num_mean_crossings = ((val_array[:-1] * val_array[1:]) < np.nanmean(list_values)).sum()  
    return [num_zero_crossings, num_mean_crossings]
 
def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [ ]:
# adapted from https://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning/
WAVELET_NAME = 'db2' 

def get_dataset_features(dataset):
    feature_vectors = []
    
    for event_eeg_index in range(0, len(dataset)):
        event_features = []
        
        for channel_index in range(0, len(dataset[event_eeg_index])):
            channel_signal = dataset[event_eeg_index][channel_index]
            print(channel_signal)
            
            coeff_list = pywt.wavedec(channel_signal, WAVELET_NAME)
            #print(coeff_list)
            for coeff in coeff_list:
                #print(coeff)
                pass
                #event_features += get_features(coeff)
                
        feature_vectors.append(event_features)
        
    X = np.array(feature_vectors)
    return X

In [ ]:
print(monster_matrix)

In [ ]:
get_dataset_features(monster_matrix)

In [ ]:
# split data using (training vs testing)
# test_size: what proportion of original data is used for test set
train_data, test_data, train_lbl, test_lbl = train_test_split(X_list, Y_list, test_size=0.20, random_state=0)

In [ ]:
def calcAccuracy(predictions, y):
    # True positives, false positives, etc.
    TP_ = np.logicaland(predictions, y)
    FP = np.logical_and(predictions, np.logicalnot(y))
    TN = np.logical_and(np.logical_not(predictions), np.logicalnot(y))
    FN = np.logical_and(np.logicalnot(predictions), y)

    TP = sum(TP)
    FP = sum(FP)
    TN = sum(TN)
    FN = sum(FN_)
    
    accuracy = (TP + TN)/(TP + FP + TN + FN)
    print('Accuracy:{}'.format(accuracy))
    
    return accuracy

In [ ]:
# perform classification using SVM
clf = svm.LinearSVC()
clf.fit(train_data, train_lbl)
y_pred = clf.predict(test_data)

# compare predictions for accuracy
accuracy = calcAccuracy(y_pred, Y_list)

# future work: we could compare accuracy linearSVC and just SVC